In [2]:
import streamlit as st
import pandas as pd
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
import keyring
from pandas import DataFrame
from langchain.embeddings import OpenAIEmbeddings
import yaml
import os
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, RetrievalQA
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentType, initialize_agent
from langchain.tools import Tool, YouTubeSearchTool
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.chat_models.openai import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate


# PARAMETERS
# Neo4j connection parameters
with open('config.yaml', 'r') as file:
    cfg = yaml.safe_load(file)
user = cfg['neo4j_user']
uri = cfg['neo4j_db_uri']

# keyring fetches from OS credential manager
pw = keyring.get_password('eastridge', user)
driver = GraphDatabase.driver(uri, auth=(user, pw))

# OpenAI Setup
os.environ['OPENAI_API_KEY'] = keyring.get_password('eastridge', 'openai')

# Similarity Search

In [3]:
embedding_provider = OpenAIEmbeddings()

patent_abstract_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url=uri,
    username=user,
    password=pw,
    index_name="embeddingsindex",
    embedding_node_property="embedding",
    text_node_property="abstract",
)



In [4]:
# k=1, most similar response
r = patent_abstract_vector.similarity_search("A system for removing an occlusive clot from a blood vessel.", k=1)
print(r)

[Document(page_content='A system for removing an occlusive clot from a blood vessel comprises a catheter and an apparatus for generating a pulsatile vacuum force to pulse the pressure gradient at a distal end of the catheter. The pulse generator may be integral with or separate from the vacuum pump. The pulse generator may be applied to a flexible tubing between the vacuum pump and the proximal end of the catheter.', metadata={'title': 'CLOT RETRIEVAL SYSTEM FOR REMOVING OCCLUSIVE CLOT FROM A BLOOD VESSEL', 'docnumber': 'US20230000510A1', 'date': neo4j.time.Date(2023, 1, 5), 'reducedEmbedding': [-0.020457884296774864, 0.015469152480363846, -0.14464735984802246, 0.03954287990927696, 0.013515464030206203, 0.0978100448846817, -0.013073090463876724, -0.048353053629398346, 0.13818566501140594, 0.10264269262552261, 0.10216636210680008, 0.01920410990715027, -0.011776513420045376, 0.08536525815725327, 0.002027187729254365, -0.020735934376716614, 0.07305675745010376, 0.011173881590366364, 0.016

In [35]:
r = patent_abstract_vector.similarity_search("An immersive VR system designed for education.", k=1000)


pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None) 

data_list = []
for document in r:
    page_content = document.page_content
    title = document.metadata['title']
    data_list.append({'Title': title})



df = pd.DataFrame(data_list)
display(df)

,Title
0,VIRTUAL REALITY SYSTEM
1,THREE DIMENSIONAL VIRTUAL AND AUGMENTED REALITY DISPLAY SYSTEM
2,SPORT TRAINING METHOD AND SYSTEM AND HEAD-MOUNTED VR DEVICE
3,VIRTUAL AND AUGMENTED REALITY SYSTEMS AND METHODS
4,AUGMENTED AND VIRTUAL REALITY DISPLAY SYSTEMS AND METHODS FOR DIAGNOSING HEALTH CONDITIONS BASED ON VISUAL FIELDS
...,...
995,LIQUID CRYSTAL DEVICE
996,APPARATUS AND METHOD FOR RENDERING A SOUND SCENE COMPRISING DISCRETIZED CURVED SURFACES
997,DISPLAY MODULE AND DISPLAY APPARATUS HAVING THE SAME
998,"IMAGING APPARATUS, CONTROL METHOD FOR IMAGING APPARATUS, AND STORAGE MEDIUM"


# Using a RetrievalQA

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models.openai import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.neo4j_vector import Neo4jVector

chat_llm = ChatOpenAI()

patent_abstract_vector = Neo4jVector.from_existing_index(
    embedding_provider,
    url=uri,
    username=user,
    password=pw,
    index_name="embeddingsindex",
    embedding_node_property="embedding",
    text_node_property="abstract",
)

embedding_provider = OpenAIEmbeddings()

retrievalQA = RetrievalQA.from_llm(
    llm=chat_llm,
    retriever=patent_abstract_vector.as_retriever(),
    verbose=True,
    return_source_documents=True
)

response = retrievalQA(f"""
You are a patent expert. You provide insights and recommendations on patents based on their content, technology, and inventors.

I am researching wearable technology for health monitoring. Can you recommend some patents that I should look into?
""")




> Entering new RetrievalQA chain...

> Finished chain.


In [9]:
if 'result' in response:
    print(response['result'])
else:
    print("No relevant result found.")

Based on your interest in wearable technology for health monitoring, here are a few patents that you may find relevant:

1. Patent Title: "Wearable device for providing therapy based on physiological parameters"
   Summary: This patent describes a system where a wearable device monitors physiological parameters of the user and triggers therapy automatically when certain thresholds are reached.
   Patent Number: [Patent Number]

2. Patent Title: "System for wearable authentication and management"
   Summary: This patent discloses a system that uses biometric data or movement signatures to identify and authenticate users of wearable devices. It also includes features like provisioning connectivity services, associating devices with the user's ecosystem, and delivering communications to the wearable device.
   Patent Number: [Patent Number]

3. Patent Title: "Wearable device with electrode unit and ventilation holes"
   Summary: This patent focuses on a specific wearable device design tha

In [10]:

technical_description = response['result'] if 'result' in response else None

input_prompt = f"Simplify the following in laymen's terms, in less than 200 tokens: {technical_description}"
simplified_response = chat_llm.predict(input_prompt)




In [38]:
simplified_response

"Here are some patents related to wearable technology for monitoring health. These patents describe different wearable devices and systems that can track and monitor various aspects of your health, such as vital signs and physiological data. They include features like sensors and remote monitoring capabilities. However, it's important to note that these are just references and you should do further research to find the patents that best match your specific interests and needs."

# Query using Natural Language

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate

graph = Neo4jGraph(
    url=uri,
    username=user,
    password=pw,
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about Patent recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.
Return only a string response of answers to the query.

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)

cypher_chain = GraphCypherQAChain.from_llm(
    chat_llm,
    graph=graph,
    cypher_prompt=cypher_generation_prompt,
    verbose=True
)




In [15]:
cypher_chain.run("What patents are connected to HAIER US APPLIANCE SOLUTIONS, INC.? Don't use arrows in the Cypher query as a way to remove directionality.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Assignee {name: 'HAIER US APPLIANCE SOLUTIONS, INC.'})-[:ASSIGNED_TO]-(d:Document)
RETURN d.title
Full Context:
[{'d.title': 'FLUID DELIVERY SYSTEM FOR A DRYER APPLIANCE'}, {'d.title': 'DRIVETRAIN FOR A MIXER'}, {'d.title': 'WASHER APPLIANCE WITH REMOVABLE AGITATOR POST HAVING TWIST LOCK MECHANISM'}, {'d.title': 'SUCTION MUFFLER FOR A RECIPROCATING COMPRESSOR'}, {'d.title': 'FLUID INFUSING SYSTEM DRIVEN BY DRYER DRUM'}, {'d.title': 'DRAINLESS CLEAR ICE MAKER FOR RECYCLING WATER USED TO MAKE CLEAR ICE'}, {'d.title': 'SYSTEM AND METHOD FOR DETECTING A TOWER POSITIONING FAULT USING A DRIVE ASSEMBLY IN AN INDOOR GARDEN CENTER'}]

> Finished chain.


'The patents connected to HAIER US APPLIANCE SOLUTIONS, INC. are as follows:\n1. FLUID DELIVERY SYSTEM FOR A DRYER APPLIANCE\n2. DRIVETRAIN FOR A MIXER\n3. WASHER APPLIANCE WITH REMOVABLE AGITATOR POST HAVING TWIST LOCK MECHANISM\n4. SUCTION MUFFLER FOR A RECIPROCATING COMPRESSOR\n5. FLUID INFUSING SYSTEM DRIVEN BY DRYER DRUM\n6. DRAINLESS CLEAR ICE MAKER FOR RECYCLING WATER USED TO MAKE CLEAR ICE\n7. SYSTEM AND METHOD FOR DETECTING A TOWER POSITIONING FAULT USING A DRIVE ASSEMBLY IN AN INDOOR GARDEN CENTER'

In [18]:
result = cypher_chain.run("Who invented this patent?  US20230002955A1")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Document {docnumber: 'US20230002955A1'})-[:INVENTED_BY]->(i:Inventor)
RETURN i.name
Full Context:
[{'i.name': 'VENKATA CHAKRADHAR RANGU'}, {'i.name': 'ALEXANDER B. LEIBMAN'}, {'i.name': 'NEELAPALA GOPINATH YADAV'}]

> Finished chain.


In [17]:
result

"I'm sorry, but I don't have the information to answer your question."

In [58]:
import io
import sys

# Redirect stdout to a string
old_stdout = sys.stdout
sys.stdout = buffer = io.StringIO()

# Run the chain
result = cypher_chain.run("Who invented this patent? US20230002955A1")

# Get everything that was printed to the console
verbose_output = buffer.getvalue()

# Reset stdout to its original state
sys.stdout = old_stdout

print(verbose_output)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Document {docnumber: 'US20230002955A1'})-[:INVENTED_BY]->(inventor:Inventor)
RETURN inventor.name
Full Context:
[{'inventor.name': 'VENKATA CHAKRADHAR RANGU'}, {'inventor.name': 'ALEXANDER B. LEIBMAN'}, {'inventor.name': 'NEELAPALA GOPINATH YADAV'}]

> Finished chain.



In [ ]:
"Can you find patents related to autonomous vehicle technology filed in the last five years?"
"I'm interested in recent innovations in solar energy storage. What are the key features of the latest patents in this field?"
"Compare the approaches of patents US20230001162A1 and US20230001061A1."
"I am researching wearable technology for health monitoring. Can you recommend some patents that I should look into?"

"Who invented the patent US20230001061A1?"
"Which patent is most similar to patent US20230001061A1?"
"Compare the approaches of patents US20230001162A1 and US20230001061A1 based on the text in their abstracts."